<a href="https://colab.research.google.com/github/MatiTaila/hack_iia/blob/ia-2003/face_detector/generar_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Montar disco de Google Drive

Para proder acceder a datos guardados en su google drive desde un notebook de colab es necesario ejecutar el siguiente código

Esto va a ser necesario en todos los notebooks de este curso.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Generar Dataset

El primer paso en cualquier algoritmo de Machine Learning es preprocesar los datos.

En particular para aprendizaje supervisado es importante generar un set de datos de entrenamiento y un set de datos de test para poder validar el desempeño de nuestro algoritmo (En general tambien se usa un tercer set de datos llamado validación).

Para generar nuestro set de datos vamos a seguir los siguientes pasos.
1. Dividir nuestros datos en un split de 90% training y 10% test.
2. Para los datos de training vamos a hacer lo siguiente:
    1. Se detecta una cara en cada imagen de training y se guarda en una variable.
    2. Se obtiene el identificador de cada persona que agregamos a la base de datos, a partir del nombre de la carpeta en que proviene el archivo.
    3. Se utiliza el identificador para guardar los datos de entrenamiento.
3. Los datos para Testing se guardan si hacer detección facial.

Es una buena práctica hacer shuffle de los datos antes de dividir en training y test pero en este caso vamos a recorrer los archivos secuenciales para que todos tengan el mismo resultado.

Para comenzar tambien vamos a visualizar los datos de entrenamiento

In [0]:
import cv2
import numpy as np
from PIL import Image
import os, glob
import shutil
import matplotlib.pyplot as plt
import random

# Definimos las carpetas y archivos especificos que vamos a utilizar en el script
DATASET_DIR = '/content/drive/My Drive/hack_iia/face_detector/dataset'
CASCADE_FILE = '/content/drive/My Drive/hack_iia/face_detector/default.xml'
TRAIN_DIR = '/content/drive/My Drive/hack_iia/face_detector/dataset/train'
TEST_DIR = '/content/drive/My Drive/hack_iia/face_detector/dataset/test'


In [0]:
## Funcion para mostrar datos de entrenamiento
def show_random_sample(train_dir,count=5):
    img_paths = glob.glob(os.path.join(train_dir,"*.jpg"))
    random_paths = random.sample(img_paths,count)
    fig, ax = plt.subplots(1,count, figsize=(20,3))
    for idx, random_path in enumerate(random_paths):
        img = Image.open(random_path)
        ax[idx].imshow(img,cmap='gray')
        ax[idx].set_xticks([])
        ax[idx].set_yticks([])
    plt.show()

### Visualización de Datos de entranmiento

In [0]:
SUBJECTS = ['jeniffer', 'brad']

#### JENNIFER ANISTON 

In [0]:
show_random_sample(os.path.join(DATASET_DIR,SUBJECTS[0]),5)

#### BRAD PITT ###

In [0]:
show_random_sample(os.path.join(DATASET_DIR,SUBJECTS[1]),5)

### PROCESAMOS LOS DATOS PARA EXTRAER CARAS

In [0]:

# Inicializaos el detector facial
detector = cv2.CascadeClassifier(CASCADE_FILE)
# Inicializaos el reconocedor facial
reconocedor = cv2.face.LBPHFaceRecognizer_create()

for subject_count, subject_name in enumerate(SUBJECTS):
    # Buscamso todas imagenes dentro de la carpeta donde esta nuestra base de datos
    image_paths = glob.glob(os.path.join(DATASET_DIR,subject_name,"*.jpg"))
    id = subject_count + 1
    
    # Para cada imagen de entrenamiento:
    for img_count, img_path in enumerate(image_paths[:170]):
        # Cargamos la imagen y la convertimos a escala de grises
        PIL_img = Image.open(img_path).convert('L')
        img_numpy = np.array(PIL_img,'uint8')
        # Detectamos las caras
        faces = detector.detectMultiScale(img_numpy, 1.2, 5,  minSize=(int(0.2*img_numpy.shape[0]), int(0.2*img_numpy.shape[1])))
        # Para cada cara agregamos a nuestras variables la cara y el id de la cara
        for (x,y,w,h) in faces:
            # guardamos la imagen capturada en la carpeta de datos de entrenamiento
            cv2.imwrite(TRAIN_DIR + "/persona." + str(id) + '.im' + str(img_count) + ".jpg", img_numpy[y:y + h, x:x + w])
    
    # Para cada imagen de testing simplemente copiamos:
    for img_count, img_path in enumerate(image_paths[170:]):
        image_name = subject_name + str(img_count) + '.jpg'
        shutil.copy(img_path, os.path.join(TEST_DIR,image_name))



### DATOS PROCESAD0S ###

In [0]:
show_random_sample(TRAIN_DIR, 10)